In [1]:
with open("q18_input.txt", "r") as fp:
    f_lines = fp.readlines()

In [2]:
class SnailParse:
    def __init__(self):
        self.s = []  # stack for number, level
        self.curr_level = 0
        
    def reset_level(self):
        self.curr_level = 0
        
    def add_level(self, new_s):
        for i in range(len(new_s)):
            new_s[i][1] += 1
        self.s += new_s
        
    def level_up(self):
        for i in range(len(self.s)):
            self.s[i][1] += 1
            self.curr_level = 1

    def add_new_c(self, c):
        if c == ",":
            pass
        elif c == "[":
            self.curr_level += 1
        elif c == "]":
            self.curr_level -= 1
        else:
            v = int(c)
            self.s.append([v, self.curr_level])

    
    def explode(self):
        i = 0
        # explode
        while i < len(self.s):
            # print(i, self.s)
            step = 1
            number, level = self.s[i]
            # explode
            if level >= 5:
                # remove i
                self.s.pop(i)
                # actually remove i + 1
                next_number, next_level  = self.s.pop(i)
                # explode right
                if i < len(self.s):
                    self.s[i][0] += next_number
                # explode left
                if i > 0:
                    self.s[i - 1][0] += number
                    # set the iterator back to check split
                    step = -1
                self.s.insert(i, [0, level - 1])
                return True
            i += 1
        return False
            
    def split(self):
        i = 0
        # explode
        while i < len(self.s):
            # print(i, self.s)
            step = 1
            number, level = self.s[i]
            # explode
            if number >= 10:
                v0, v1 = number // 2, number - number // 2
                self.s[i] = [v0, level + 1]
                self.s.insert(i + 1, [v1, level + 1])
                step = 0
                return True
            i += 1
        return False
        
    def check(self):
        # THIS IS SO CONFUSING
        # HATE IT
        flag = True
        while flag:
            flag = self.explode()
            if not flag:
                flag = self.split()
                
    def magnitude(self):
        """REALIZED I NEED TO KEEP TRACK OF []"""

In [3]:
# master_p = SnailParse()
# parsed_lines = []
# for line in f_lines:
#     # print(line)
#     p = SnailParse()
#     for c in line.strip():
#         p.add_new_c(c)
#     p.explode()
#     # print(p.s)
#     parsed_lines.append(p)

# for p in parsed_lines:
#     master_p.add_level(p.s)
#     print(master_p.s)
#     master_p.explode()
# print(master_p.s)

In [4]:
master_p = SnailParse()
for n_l, line in enumerate(f_lines):
    for c in line.strip():
        master_p.add_new_c(c)
    master_p.reset_level()
    #  print("-->", master_p.s)
    master_p.check()
    if n_l != len(f_lines) - 1:
        master_p.level_up()
print(master_p.s)

[[7, 4], [7, 4], [7, 4], [0, 4], [7, 4], [8, 4], [8, 4], [8, 4], [6, 2], [9, 2]]


# GIVE UP

In [5]:
def explode_num(num):
    def add_to_leftmost(num, val):
        if isinstance(num, int):
            return num + val
        a, b = num
        return [add_to_leftmost(a, val), b]

    def add_to_rightmost(num, val):
        if isinstance(num, int):
            return num + val
        a, b = num
        return [a, add_to_rightmost(b, val)]

    # Returns exploded, left_exp, new_num, right_exp
    def impl(num, depth):
        if isinstance(num, int):
            return False, None, num, None

        a, b = num
        if depth == 4:
            return True, a, 0, b

        exploded, left_exp, a, right_exp = impl(a, depth+1)
        if exploded:
            if right_exp is not None:
                b = add_to_leftmost(b, right_exp)
                right_exp = None
        else:
            exploded, left_exp, b, right_exp = impl(b, depth+1)

            if left_exp is not None:
                a = add_to_rightmost(a, left_exp)
                left_exp = None

        return exploded, left_exp, [a, b], right_exp

    exploded, _, new_num, _ = impl(num, 0)
    return new_num if exploded else None

def split_num(num):
    if isinstance(num, int):
        if num > 9:
            d, m = divmod(num, 2)
            return [d, d + m]
        return None

    a, b = num

    new_a = split_num(a)
    if new_a is not None:
        return [new_a, b]

    new_b = split_num(b)
    if new_b is not None:
        return [a, new_b]

    return None

def reduce_num(num):
    while True:
        new_num = explode_num(num)
        if new_num is not None:
            num = new_num
            continue

        new_num = split_num(num)
        if new_num is not None:
            num = new_num
            continue

        return num

def magnitude(num):
    if isinstance(num, int):
        return num

    a, b = num
    return 3*magnitude(a) + 2*magnitude(b)

def part1(s):
    nums = list(map(reduce_num, map(eval, s)))

    n = nums[0]
    for item in nums[1:]:
        n = reduce_num([n, item])

    answer = magnitude(n)

    print(f'The answer to part one is {answer}')

def part2(s):
    nums = list(map(reduce_num, map(eval, s)))

    answer = max(magnitude(reduce_num([a, b]))
                 for ia, a in enumerate(nums)
                 for ib, b in enumerate(nums)
                 if ia != ib)

    print(f'The answer to part two is {answer}')

In [6]:
part1(f_lines)

The answer to part one is 2541


In [7]:
part2(f_lines)

The answer to part two is 4647
